In [1]:
import utils as ut

In [2]:
import ibllib.io.raw_data_loaders as raw

In [7]:
import pandas as pd

# Putting Multiple Days of Data into a Single npz File

### Getting Training Session Paths
* get user choice of a folder for that mouse 
* generate dict w/ the file paths of each day as keys
* for each key, generate a list of the file paths contained w/in that folder  

### Getting the number of trials in each day 
* Run the same data processing as was used on a single file (above) on each file in the list of file paths for each day
    * create an empty list for day lengths
    * get the number of trials in each day and append it to the list

In [3]:
# get user choice of folder path for a particular mouse
mouseFolderPath = ut.selectFolder()

# generate a list of the file paths of the subfolders to mouseFolderPath
mouseSessionsPaths_lst = ut.getSubfolders(mouseFolderPath)
mouseSessionsPaths_lst # should contain a list of folders titled by date 

2023-02-10 17:35:00.731 python[22416:17393683] +[CATransaction synchronize] called within transaction
2023-02-10 17:35:00.895 python[22416:17393683] +[CATransaction synchronize] called within transaction


['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-10-10',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-10-18',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-10-11',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-14',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-13',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-12',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-15',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-11-21',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-08',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-30',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-06',
 '/Users/joshuadamato/Documents/adesnikLab/sampleBehav

In [22]:
mouseFolderPath
# there are multiple slash symbols in mouseFolderPath, so we need to find the last one and then take the substring after that last slash symbol
mouseName = mouseFolderPath[mouseFolderPath.rfind('/')+1:]
mouseName

In [4]:
dailySessions_dict = {}
for sessionPath in mouseSessionsPaths_lst:
    dailySessions_dict[sessionPath[-10:]] = ut.getSubfolders(sessionPath)

dailySessions_dict

{'2022-10-10': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-10-10/001'],
 '2022-10-18': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-10-18/001',
  '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-10-18/002'],
 '2022-10-11': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-10-11/001'],
 '2022-09-14': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-14/001'],
 '2022-09-13': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-13/001'],
 '2022-09-12': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-12/001'],
 '2022-09-15': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-15/001',
  '/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData/MBOT53_013/2022-09-15/002'],
 '2022-11-21': ['/Users/joshuadamato/Documents/adesnikLab/sampleBehaviorData

In [5]:
# function for changing values to binary(ish) format
def changeVals(x):
    if x < 0:
        return 1
    else:
        return 2

In [13]:
concat_df = pd.DataFrame()
concat_df['name'] = mouseName
dayLength = []
for key in dailySessions_dict.keys():
    for session in dailySessions_dict[key]:
        data = raw.load_data(session)
        trialinfo_df = raw.load_encoder_trial_info(session)

        # getting a list of correct trials 
        correcttrials_lst = [d['trial_correct'] for d in data] 

        # correcting for the possibility of the last trial not being saved in the encoder_trial_info file 
        diff = len(trialinfo_df) - len(correcttrials_lst)
        if diff:
            print(f'warning, {diff} trial\'s is/are missing from the data dictionary')
            trialinfo_df = trialinfo_df[:-diff]

        assert (len(trialinfo_df) == len(correcttrials_lst)), 'trialinfo_df and correcttrials_lst are not the same length' # COMMENTED OUT FOR NOW
        if len(trialinfo_df) > len(correcttrials_lst):
            trialinfo_df[:len(correcttrials_lst)+1]
        elif len(correcttrials_lst) > len(trialinfo_df):
            correcttrials_lst[:len(trialinfo_df)+1]

        dayLength.append(len(correcttrials_lst))

        # injecting trial-wise data into the trialinfo_df, changing values to 1 or 2 to conform to psytrack's format
        """correct -- the accurarcy of the choice (1 = correct, 2 = incorrect)"""
        trialinfo_df.loc[:, 'correct'] = [1 if x else 0 for x in correcttrials_lst]

        # changing stim angle value to 0 (-left) or 1 (+right) 
        trialinfo_df['answer'] = trialinfo_df['stim_angle'].apply(changeVals)

        # adds a column for when the wheel is turned to the left:
        trialinfo_df["LeftTurn"] = ((trialinfo_df["stim_angle"] < 0) & (trialinfo_df["correct"] == 1)) | ((trialinfo_df["stim_angle"] >= 0) & (trialinfo_df["correct"] == 0))
        trialinfo_df["LeftTurn"] = trialinfo_df["LeftTurn"].astype(int)

        # adds signed contrast
        trialinfo_df.loc[:,'signedContrast'] = trialinfo_df.stim_contrast.values*trialinfo_df.stim_pos_init.values

warning, 1 trial's is/are missing from the data dictionary
warning, 1 trial's is/are missing from the data dictionary
warning, -57 trial's is/are missing from the data dictionary


ValueError: Length of values (437) does not match length of index (57)